In [ ]:
# default_exp data.seq2seq.language_modeling

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# data.seq2seq.language_modeling

> This module contains the bits required to use the fastai DataBlock API and/or mid-level data processing pipelines to organize your data for causal and masked language modeling tasks. This includes things like training BERT from scratch or fine-tuning a particular pre-trained LM on your own corpus.

**This is currently a work in progress** - You've been warned : )

In [ ]:
#export
import torch, pdb
from transformers import *
from fastai.text.all import *

from blurr.utils import *
from blurr.data.core import *
from blurr.data.seq2seq.core import *

from abc import ABC, abstractmethod

logging.set_verbosity_error()

In [ ]:
#hide
import pdb

from nbdev.showdoc import *
from fastcore.test import *

from fastai import __version__ as fa_version
from torch import __version__ as pt_version
from transformers import __version__ as hft_version

print(f'Using pytorch {pt_version}')
print(f'Using fastai {fa_version}')
print(f'Using transformers {hft_version}')

Using pytorch 1.7.1
Using fastai 2.3.1
Using transformers 4.5.1


In [ ]:
#cuda
torch.cuda.set_device(1)
print(f'Using GPU #{torch.cuda.current_device()}: {torch.cuda.get_device_name()}')

Using GPU #1: GeForce GTX 1080 Ti


In [ ]:
wiki_path = untar_data(URLs.WIKITEXT_TINY)
wiki_path.ls()

(#2) [Path('/home/wgilliam/.fastai/data/wikitext-2/train.csv'),Path('/home/wgilliam/.fastai/data/wikitext-2/test.csv')]

In [ ]:
train_df = pd.read_csv(wiki_path/'train.csv', header=None)
valid_df = pd.read_csv(wiki_path/'test.csv', header=None)

print(len(train_df), len(valid_df))
train_df.head()

615 47


,0
0,"\n = 2013 – 14 York City F.C. season = \n \n The 2013 – 14 season was the <unk> season of competitive association football and 77th season in the Football League played by York City Football Club , a professional football club based in York , North Yorkshire , England . Their 17th @-@ place finish in 2012 – 13 meant it was their second consecutive season in League Two . The season ran from 1 July 2013 to 30 June 2014 . \n Nigel Worthington , starting his first full season as York manager , made eight permanent summer signings . By the turn of the year York were only above the relegation z..."
1,"\n = Big Boy ( song ) = \n \n "" Big Boy "" <unk> "" I 'm A Big Boy Now "" was the first single ever recorded by the Jackson 5 , which was released by Steeltown Records in January 1968 . The group played instruments on many of their Steeltown compositions , including "" Big Boy "" . The song was neither a critical nor commercial success , but the Jackson family were delighted with the outcome nonetheless . \n The Jackson 5 would release a second single with Steeltown Records before moving to Motown Records . The group 's recordings at Steeltown Records were thought to be lost , but they were re..."
2,"\n = The Remix ( Lady Gaga album ) = \n \n The Remix is a remix album by American recording artist Lady Gaga . Released in Japan on March 3 , 2010 , it contains remixes of the songs from her first studio album , The Fame ( 2008 ) , and her third extended play , The Fame Monster ( 2009 ) . A revised version of the track list was prepared for release in additional markets , beginning with Mexico on May 3 , 2010 . A number of recording artists have produced the songs , including Pet Shop Boys , Passion Pit and The Sound of Arrows . The remixed versions feature both uptempo and <unk> composit..."
3,"\n = New Year 's Eve ( Up All Night ) = \n \n "" New Year 's Eve "" is the twelfth episode of the first season of the American comedy television series Up All Night . The episode originally aired on NBC in the United States on January 12 , 2012 . It was written by Erica <unk> and was directed by Beth McCarthy @-@ Miller . The episode also featured a guest appearance from Jason Lee as Chris and Reagan 's neighbor and Ava 's boyfriend , Kevin . \n During Reagan ( Christina Applegate ) and Chris 's ( Will <unk> ) first New Year 's Eve game night , Reagan 's competitiveness comes out causing Ch..."
4,"\n = Geopyxis carbonaria = \n \n Geopyxis carbonaria is a species of fungus in the genus Geopyxis , family <unk> . First described to science in 1805 , and given its current name in 1889 , the species is commonly known as the charcoal loving elf @-@ cup , dwarf <unk> cup , <unk> <unk> cup , or pixie cup . The small , <unk> @-@ shaped fruitbodies of the fungus are reddish @-@ brown with a whitish fringe and measure up to 2 cm ( 0 @.@ 8 in ) across . They have a short , tapered stalk . Fruitbodies are commonly found on soil where brush has recently been burned , sometimes in great numbers ...."


In [ ]:
train_df['is_valid'] = False
valid_df['is_valid'] = True

df = pd.concat([train_df, valid_df])
df.head()

,0,is_valid
0,"\n = 2013 – 14 York City F.C. season = \n \n The 2013 – 14 season was the <unk> season of competitive association football and 77th season in the Football League played by York City Football Club , a professional football club based in York , North Yorkshire , England . Their 17th @-@ place finish in 2012 – 13 meant it was their second consecutive season in League Two . The season ran from 1 July 2013 to 30 June 2014 . \n Nigel Worthington , starting his first full season as York manager , made eight permanent summer signings . By the turn of the year York were only above the relegation z...",False
1,"\n = Big Boy ( song ) = \n \n "" Big Boy "" <unk> "" I 'm A Big Boy Now "" was the first single ever recorded by the Jackson 5 , which was released by Steeltown Records in January 1968 . The group played instruments on many of their Steeltown compositions , including "" Big Boy "" . The song was neither a critical nor commercial success , but the Jackson family were delighted with the outcome nonetheless . \n The Jackson 5 would release a second single with Steeltown Records before moving to Motown Records . The group 's recordings at Steeltown Records were thought to be lost , but they were re...",False
2,"\n = The Remix ( Lady Gaga album ) = \n \n The Remix is a remix album by American recording artist Lady Gaga . Released in Japan on March 3 , 2010 , it contains remixes of the songs from her first studio album , The Fame ( 2008 ) , and her third extended play , The Fame Monster ( 2009 ) . A revised version of the track list was prepared for release in additional markets , beginning with Mexico on May 3 , 2010 . A number of recording artists have produced the songs , including Pet Shop Boys , Passion Pit and The Sound of Arrows . The remixed versions feature both uptempo and <unk> composit...",False
3,"\n = New Year 's Eve ( Up All Night ) = \n \n "" New Year 's Eve "" is the twelfth episode of the first season of the American comedy television series Up All Night . The episode originally aired on NBC in the United States on January 12 , 2012 . It was written by Erica <unk> and was directed by Beth McCarthy @-@ Miller . The episode also featured a guest appearance from Jason Lee as Chris and Reagan 's neighbor and Ava 's boyfriend , Kevin . \n During Reagan ( Christina Applegate ) and Chris 's ( Will <unk> ) first New Year 's Eve game night , Reagan 's competitiveness comes out causing Ch...",False
4,"\n = Geopyxis carbonaria = \n \n Geopyxis carbonaria is a species of fungus in the genus Geopyxis , family <unk> . First described to science in 1805 , and given its current name in 1889 , the species is commonly known as the charcoal loving elf @-@ cup , dwarf <unk> cup , <unk> <unk> cup , or pixie cup . The small , <unk> @-@ shaped fruitbodies of the fungus are reddish @-@ brown with a whitish fringe and measure up to 2 cm ( 0 @.@ 8 in ) across . They have a short , tapered stalk . Fruitbodies are commonly found on soil where brush has recently been burned , sometimes in great numbers ....",False


## Abstract LMStrategy Class

In [ ]:
#export
class LMStrategy(ABC):
    def __init__(self, hf_tokenizer, ignore_token_id=CrossEntropyLossFlat().ignore_index):
        store_attr(['hf_tokenizer', 'ignore_token_id'])
         
    @abstractmethod
    def build_inputs_targets(self, samples):
        pass
    
    # utility methods
    def _get_random_token_id(self,n):
        return random.sample(list(self.hf_tokenizer.get_vocab().values()), n)

## Causal LM

In [ ]:
model_cls = AutoModelForCausalLM

pretrained_model_name = "gpt2"
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(pretrained_model_name, model_cls=model_cls)

# some tokenizers like gpt and gpt2 do not have a pad token, so we add it here mainly for the purpose
# of setting the "labels" key appropriately (see below)
if (hf_tokenizer.pad_token is None): hf_tokenizer.pad_token = '[PAD]'

hf_tokenizer.pad_token, hf_tokenizer.pad_token_id

Using pad_token, but it is not set yet.


('[PAD]', 50256)

In [ ]:
#export
class CausalLMStrategy(LMStrategy):
    
    def build_inputs_targets(self, samples):
        updated_samples = []
        for s in samples:
            s[0]['labels'] = s[0]['input_ids'].clone()
            s[0]['labels'][s[0]['labels'] == self.hf_tokenizer.pad_token_id] = self.ignore_token_id            
            targ_ids = s[0]['input_ids'].clone()

            updated_samples.append((s[0], targ_ids))
        
        return updated_samples

In [ ]:
#export
class HF_LMBeforeBatchTransform(HF_BeforeBatchTransform):
    def __init__(self, hf_arch, hf_config, hf_tokenizer, hf_model, lm_strategy_cls:LMStrategy=CausalLMStrategy,
                 max_length=None, padding=True, truncation=True, is_split_into_words=False, 
                 ignore_token_id = CrossEntropyLossFlat().ignore_index,
                 tok_kwargs={}, text_gen_kwargs={}, **kwargs):
                 
        super().__init__(hf_arch, hf_config, hf_tokenizer, hf_model,
                         max_length=max_length, padding=padding, truncation=truncation, 
                         is_split_into_words=is_split_into_words,
                         tok_kwargs=tok_kwargs.copy(), **kwargs)
        
        self.lm_strategy = lm_strategy_cls(hf_tokenizer=hf_tokenizer, ignore_token_id=ignore_token_id)
        self.text_gen_kwargs, self.ignore_token_id = text_gen_kwargs, ignore_token_id
        
    def encodes(self, samples):  
        # because no target is specific in CLM, fastai will duplicate the inputs (which is just the raw text)
        samples = super().encodes(samples)
        if (len(samples[0]) == 1): return samples
        
        return self.lm_strategy.build_inputs_targets(samples)

Our `HF_CausalLMBeforeBatchTransform` allows us to update the input's `labels` and our targets appropriately given a causal LM task. 

The `labels` argument allows you to forgo calculating the loss yourself by letting huggingface return it for you should you choose to do that. Padding tokens are set to -100 by default (e.g., `CrossEntropyLossFlat().ignore_index`) and prevent cross entropy loss from considering token prediction for tokens it should ... i.e., the padding tokens. For more information on the meaning of this argument, see the [huggingface glossary entry for "Labels"](https://huggingface.co/transformers/glossary.html#labels)

In [ ]:
before_batch_tfm = HF_LMBeforeBatchTransform(hf_arch, hf_config, hf_tokenizer, hf_model)
blocks = (HF_Seq2SeqBlock(before_batch_tfm=before_batch_tfm), noop)

dblock = DataBlock(blocks=blocks, get_x=ColReader(0), splitter=ColSplitter(col='is_valid'))

In [ ]:
dls = dblock.dataloaders(df, bs=4)

In [ ]:
b = dls.one_batch()

In [ ]:
b[0]['input_ids'].shape, b[0]['labels'].shape, b[1].shape

(torch.Size([4, 1024]), torch.Size([4, 1024]), torch.Size([4, 1024]))

In [ ]:
explode_types(b)

{tuple: [dict, torch.Tensor]}

In [ ]:
dls.show_batch(dataloaders=dls, max_n=2, trunc_at=500)

## Masked LM

In [ ]:
model_cls = AutoModelForMaskedLM

pretrained_model_name = "bert-base-uncased"
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(pretrained_model_name, model_cls=model_cls)

# some tokenizers like gpt and gpt2 do not have a pad token, so we add it here mainly for the purpose
# of setting the "labels" key appropriately (see below)
if (hf_tokenizer.pad_token is None): hf_tokenizer.pad_token = '[PAD]'

hf_tokenizer.pad_token, hf_tokenizer.pad_token_id

('[PAD]', 0)

We need a new input type for MLM tasks, particularly for `Learner.show_batch` and `Learner.show_results`

In [ ]:
#export
class HF_MLMInput(HF_BaseInput): pass

The BERT-style masking strategy

In [ ]:
#export
class BertMLMStrategy(LMStrategy):
    def __init__(self, hf_tokenizer, ignore_token_id=CrossEntropyLossFlat().ignore_index):
        super().__init__(hf_tokenizer, ignore_token_id)
         
        vocab = hf_tokenizer.get_vocab()
        self.dnm_tok_ids = [vocab[tok]  for tok in list(hf_tokenizer.special_tokens_map.values()) 
                            if vocab[tok] != hf_tokenizer.mask_token_id]
        
    def build_inputs_targets(self, samples):
        updated_samples = []
        for s in samples:
            # mask the input_ids
            masked_input_ids = s[0]['input_ids'].clone()
            
            # we want to mask 15% of the non-special tokens(e.g., special tokens inclue [CLS], [SEP], etc...)
            idxs = torch.randperm(len(masked_input_ids))
            total_masked_idxs = int(len(idxs) * .15)
            
            # of the 15% for masking, replace 80% with [MASK] token, 10% with random token, and 10% with correct token
            n_mask_idxs = int(total_masked_idxs * .8)
            n_rnd_idxs = int(total_masked_idxs * .1)
            
            # we only want non-special tokens
            mask_idxs = [ idx for idx in idxs if masked_input_ids[idx] not in self.dnm_tok_ids ][:total_masked_idxs]
            
            # replace 80% with [MASK]
            if (n_mask_idxs > 0 and  len(mask_idxs) >= n_mask_idxs):
                masked_input_ids[[mask_idxs[:n_mask_idxs]]] = self.hf_tokenizer.mask_token_id
             
            # replace 10% with a random token
            if (n_rnd_idxs > 0 and  len(mask_idxs) >= (n_mask_idxs + n_rnd_idxs)):
                rnd_tok_ids = self._get_random_token_id(n_rnd_idxs)
                masked_input_ids[[mask_idxs[n_mask_idxs:(n_mask_idxs + n_rnd_idxs)]]] = tensor(rnd_tok_ids)
            
            # ignore padding when calculating the loss
            lbls = s[0]['input_ids'].clone()
            lbls[[[idx for idx in idxs if idx not in mask_idxs]]] = self.ignore_token_id
            
            # update the inputs to use our masked input_ids and labels; set targ_ids = labels (will use when
            # we calculate the loss ourselves)
            s[0]['input_ids'] = masked_input_ids
            s[0]['labels'] = lbls
            targ_ids = lbls.clone()
            
            updated_samples.append((s[0], targ_ids))
        
        return updated_samples

In [ ]:
before_batch_tfm = HF_LMBeforeBatchTransform(hf_arch, hf_config, hf_tokenizer, hf_model, 
                                             lm_strategy_cls=BertMLMStrategy)

blocks = (HF_Seq2SeqBlock(before_batch_tfm=before_batch_tfm, input_return_type=HF_MLMInput), noop)

dblock = DataBlock(blocks=blocks, get_x=ColReader(0), splitter=ColSplitter(col='is_valid'))

In [ ]:
dls = dblock.dataloaders(df, bs=4)

In [ ]:
b = dls.one_batch()
b[0]['input_ids'].shape, b[0]['labels'].shape, b[1].shape

(torch.Size([4, 512]), torch.Size([4, 512]), torch.Size([4, 512]))

In [ ]:
b[0]['input_ids'][0][:20], b[0]['labels'][0][:20], b[1][0][:20]

(tensor([  101,  1027,  3960,  7758,  1027, 11951,  7758,  1006,  1013,  1026,
          4895,  2243,  1028,  1013,  1025,   103,  2728,  5297, 27946,  1010],
        device='cuda:1'),
 tensor([-100, -100, -100, -100, -100, 3960, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, 2141, -100, -100, -100, -100], device='cuda:1'),
 tensor([-100, -100, -100, -100, -100, 3960, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, 2141, -100, -100, -100, -100], device='cuda:1'))

In [ ]:
explode_types(b)

{tuple: [dict, torch.Tensor]}

In [ ]:
#export
@typedispatch
def show_batch(x:HF_MLMInput, y, samples, dataloaders, ctxs=None, max_n=6, trunc_at=None, **kwargs):  
    # grab our tokenizer and ignore token to decode
    hf_before_batch_tfm = get_blurr_tfm(dataloaders.before_batch)
    hf_tokenizer = hf_before_batch_tfm.hf_tokenizer
    ignore_token_id = hf_before_batch_tfm.ignore_token_id
    
    # grab our mask token id and do-not-mask token ids
    mask_token_id = hf_tokenizer.mask_token_id
    dnm_tok_ids = hf_before_batch_tfm.lm_strategy.dnm_tok_ids
    
    res = L()
    for s in samples:
        # exclue dnm tokens from input
        inps = [ hf_tokenizer.decode(tok_id) 
                if (tok_id == mask_token_id or s[1][idx] == ignore_token_id) 
                else f'[{hf_tokenizer.decode(tok_id)}]' 
                for idx, tok_id in enumerate(s[0]) if (tok_id not in dnm_tok_ids) ]
        
        # replaced masked tokens with "[{actual_token}]"
        trgs = [ hf_tokenizer.decode(s[0][idx]) 
                if (tok_id == ignore_token_id) 
                else f'[{hf_tokenizer.decode(tok_id)}]'
                for idx, tok_id in enumerate(s[1]) if (s[0][idx] not in dnm_tok_ids) ]
  
        res.append((' '.join(inps[:trunc_at]).strip(), ' '.join(trgs[:trunc_at]).strip()))
    
    display_df(pd.DataFrame(res, columns=['text', 'target'])[:max_n])
    return ctxs

In [ ]:
dls.show_batch(dataloaders=dls, max_n=2, trunc_at=250)

,text,target
0,"= [graded] dylan = [MASK] dylan ( / < un ##k > / ; born [robert] allen zimmerman , may 24 , 1941 ) is an american singer @ [-] [MASK] songwriter , artist and writer . he has been influential in popular music and [MASK] for more than five decades . [MASK] of [MASK] most celebrated work dates from [completion] 1960s when his songs chronicle ##d social unrest , [balance] dylan rep ##udi ##ated suggestions [MASK] [MASK] [MASK] he was a spokesman for his generation . nevertheless , early songs such as "" blow ##in ' in [the] wind "" and "" the times they [MASK] a @ [MASK] @ < un ##k > ' "" became anthem [MASK] for the american [MASK] [MASK] and anti @ - @ war movements [MASK] after he [left] his initial base in the [MASK] folk music revival , his [MASK] @ - @ minute single "" like a rolling stone "" altered the [MASK] of popular music [MASK] 1965 . his mid @ - @ 1960s recordings , backed by rock musicians , [MASK] [MASK] [MASK] end of the united states music charts while [MASK] attracting < [MASK] ##k > and criticism from others [MASK] the [MASK] movement . dylan ' s lyrics have incorporated [MASK] political , social [MASK] philosophical , and literary influences . they def ##ied existing pop music [MASK] and appealed to the bu ##rgeon ##ing [MASK] ##culture . [MASK] [MASK] by the performances of little [MASK] and","= [bob] dylan = [bob] dylan ( / < un ##k > / ; born [robert] allen zimmerman , may 24 , 1941 ) is an american singer @ [-] [@] songwriter , artist and writer . he has been influential in popular music and [culture] for more than five decades . [much] of [his] most celebrated work dates from [the] 1960s when his songs chronicle ##d social unrest , [although] dylan rep ##udi ##ated suggestions [from] [journalists] [that] he was a spokesman for his generation . nevertheless , early songs such as "" blow ##in ' in [the] wind "" and "" the times they [are] a @ [-] @ < un ##k > ' "" became anthem [##s] for the american [civil] [rights] and anti @ - @ war movements [.] after he [left] his initial base in the [american] folk music revival , his [six] @ - @ minute single "" like a rolling stone "" altered the [range] of popular music [in] 1965 . his mid @ - @ 1960s recordings , backed by rock musicians , [reached] [the] [top] end of the united states music charts while [also] attracting < [un] ##k > and criticism from others [in] the [folk] movement . dylan ' s lyrics have incorporated [various] political , social [,] philosophical , and literary influences . they def ##ied existing pop music [conventions] and appealed to the bu ##rgeon ##ing [counter] ##culture . [initially] [inspired] by the performances of little [richard] and"
1,"= george [calvert] , 1st baron baltimore = george calvert , 1st baron baltimore ( < un ##k > [screenings] 15 april 1632 [)] was an english politician and < un ##k > . he achieved domestic political [MASK] as a [MASK] of parliament and later secretary of state under king james i . he lost much of his political power after his support for a failed marriage alliance between prince charles and the spanish house of habsburg royal family . rather [MASK] continue [MASK] politics , he resigned all of his [MASK] offices in [1625] except for his position [MASK] [MASK] privy council and declared his catholicism publicly . he [was] [MASK] [MASK] baltimore in the irish peerage upon his resignation . baltimore [MASK] was located in county long ##ford , ireland . calvert [MASK] an interest in [MASK] british colonization [MASK] the americas , at first for commercial reasons and later to create a refuge for english [MASK] . he became the [MASK] [shelf] avalon , [MASK] first sustained english settlement on the southeastern peninsula on the [MASK] of newfoundland ( off the eastern [MASK] of [MASK] canada ) . < un ##k > by its [MASK] and sometimes in ##hos ##pit ##able climate and the [MASK] un ##k > of the settlers , sir george looked for a more suitable spot further south and sought a new royal charte

## Cleanup

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.ipynb.
Converted 01_data-core.ipynb.
Converted 01a_data-token-classification.ipynb.
Converted 01b_data-question-answering.ipynb.
Converted 01za_data-seq2seq-core.ipynb.
Converted 01zb_data-seq2seq-language-modeling.ipynb.
Converted 01zc_data-seq2seq-summarization.ipynb.
Converted 01zd_data-seq2seq-translation.ipynb.
Converted 02_modeling-core.ipynb.
Converted 02a_modeling-token-classification.ipynb.
Converted 02b_modeling-question-answering.ipynb.
Converted 02za_modeling-seq2seq-core.ipynb.
Converted 02zb_modeling-seq2seq-language-modeling.ipynb.
Converted 02zc_modeling-seq2seq-summarization.ipynb.
Converted 02zc_modeling-seq2seq-translation.ipynb.
Converted 99a_examples-multilabel.ipynb.
Converted index.ipynb.
